In [38]:
conda install -c conda-forge cudatoolkit=11.8.0
python3 -m pip install nvidia-cudnn-cu11==8.6.0.163 tensorflow==2.12.*
mkdir -p $CONDA_PREFIX/etc/conda/activate.d
echo 'CUDNN_PATH=$(dirname $(python -c "import nvidia.cudnn;print(nvidia.cudnn.__file__)"))' >> $CONDA_PREFIX/etc/conda/activate.d/env_vars.sh
echo 'export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$CONDA_PREFIX/lib/:$CUDNN_PATH/lib' >> $CONDA_PREFIX/etc/conda/activate.d/env_vars.sh
source $CONDA_PREFIX/etc/conda/activate.d/env_vars.sh
# Verify install:
python3 -c "import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))"

SyntaxError: invalid syntax (<ipython-input-38-0cf7086f26b4>, line 1)

In [42]:
import pandas as pd
import numpy as np
import seaborn as sns
import pylab as pl
import matplotlib.pyplot as plt

import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

In [2]:
Data = pd.read_csv('HIV-Dataset.csv')

In [3]:
data = Data.to_numpy()

print(data[0:3])

[[ 29.944271 793.4583    30.834505   1.         3.         0.
    0.         3.         5.         0.         0.         1.
    1.         0.         0.      ]
 [ 29.24198  467.4189    30.35598    1.         3.         0.
    0.         3.         5.         0.         0.         0.
    1.         0.         1.      ]
 [ 28.748991 465.12485   30.40532    1.         3.         0.
    0.         3.         5.         0.         0.         0.
    1.         0.         2.      ]]


TypeError: 'tuple' object cannot be interpreted as an integer

In [4]:
#search = np.array([]).reshape(int(len(data)/60) * 36, 8)

total_size = int(len(data)/60) * 36
print(total_size)

search = np.zeros((total_size, 12, 11))
demographic = np.zeros((total_size, 3))
outcome = np.zeros((total_size, 2))
#search = np.concatenate((search, [data[0:12, 0]]))

320976


In [5]:
for patient in range(int(len(data)/60)):
    # Calculate first time stamp position
    old_pos = patient*60
    
    # Store demographic information as one-off
    demo = [data[old_pos, 13], data[old_pos, 3], data[old_pos, 4]]
    
    
    for month in range(60):
        # Calculate positions
        new_pos = (patient*36)+month
        old_pos = (patient*60)+month
        
        if month < 36:
            # Store demographic
            demographic[new_pos] = demo
        
            # Add new input for next 12 months
            search[new_pos, :, 0:3] = data[old_pos:old_pos+12, 0:3]
            search[new_pos, :, 3:11] = data[old_pos:old_pos+12, 5:13]
        
            # Review change in drug compared to last value of previous month
            if (data[old_pos+12, 5:10] == data[old_pos+24, 5:10]).all():
                    outcome[new_pos, 0] = 1
        
        if month > 12:
            # Review for non-adherence
            if data[old_pos, 12] == 0.0:
                outcome[new_pos-13:new_pos-1, 1] = 1

In [44]:
ds = tf.keras.utils.timeseries_dataset_from_array(
      data=search,
      targets=outcome[:, 0],
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=True,
      batch_size=32,)


AttributeError: module 'tensorflow.python.keras.api._v1.keras.utils' has no attribute 'timeseries_dataset_from_array'

In [24]:
(Data.columns[0:3]).append(Data.columns[5:13])

Index(['VL', 'CD4', 'Rel CD4', 'Base Drug Combo', 'Comp. INI', 'Comp. NNRTI',
       'Extra PI', 'Extra pk-En', 'VL (M)', 'CD4 (M)', 'Drug (M)'],
      dtype='object')

In [43]:
col = np.array([])
for i in range(12):
    col = np.append(col, (Data.columns[0:3] + "_" + str(i)).append(Data.columns[5:13] + "_" + str(i)))


final = pd.concat([
    pd.DataFrame(demographic, columns=['PatientID', 'Gender', 'Ethnicity']),
    pd.DataFrame(search.reshape(total_size, 12 * 11), columns = col),
    pd.DataFrame(outcome, columns=['Drug Changed', "Non Adherent"])],
    axis = 1)

Each row has:

 - 3 identifiers (PatientID, Gender and Ethnicity)

 - 12 months of 11 features (132 in total)

 - 2 outcomes
   - Drug changed at end of next 12 months
   - Non adherent for any period in next 12 months

Which adds up to 137 columns per row

In [26]:
final.head()

,PatientID,Gender,Ethnicity,VL_0,CD4_0,Rel CD4_0,Base Drug Combo_0,Comp. INI_0,Comp. NNRTI_0,Extra PI_0,...,Base Drug Combo_11,Comp. INI_11,Comp. NNRTI_11,Extra PI_11,Extra pk-En_11,VL (M)_11,CD4 (M)_11,Drug (M)_11,Drug Changed,Non Adherent
0,0.0,1.0,3.0,29.944271,793.45830,30.834505,0.0,0.0,3.0,5.0,...,0.0,0.0,3.0,5.0,0.0,0.0,0.0,1.0,1.0,0.0
1,0.0,1.0,3.0,29.241980,467.41890,30.355980,0.0,0.0,3.0,5.0,...,0.0,0.0,3.0,5.0,0.0,0.0,0.0,1.0,1.0,0.0
2,0.0,1.0,3.0,28.748991,465.12485,30.405320,0.0,0.0,3.0,5.0,...,0.0,0.0,3.0,5.0,0.0,0.0,0.0,1.0,1.0,0.0
3,0.0,1.0,3.0,28.101835,692.00690,30.248816,0.0,0.0,3.0,5.0,...,0.0,0.0,3.0,5.0,0.0,0.0,0.0,1.0,1.0,0.0
4,0.0,1.0,3.0,28.813837,641.75714,29.944712,0.0,0.0,3.0,5.0,...,0.0,0.0,3.0,5.0,0.0,0.0,0.0,1.0,1.0,0.0
